In [1]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import numpy as np
from math import ceil
import copy

from sim_lib.simulation import run_simulation
from sim_lib.graph_create import stochastic_block_model, powerlaw_dist_time
from sim_lib.sim_strategies import EvenAlloc
import sim_lib.util as util

init_notebook_mode(connected=True)

In [2]:
# Constants
comm_size = 100
ncomm = 4
sbm_tot_n = comm_size * ncomm
in_ep = np.log(comm_size) / comm_size
out_ep = 1 / (sbm_tot_n ** 2)

In [3]:
def create_sbm(p, r):
    return stochastic_block_model(ncomm, comm_size,
                                  in_ep, out_ep, p, r)

In [4]:
def plot_util_comp(given_utils, r):
    fig = go.Figure()

    #Add averages trace
    p_vals = [ p for (p, uts) in given_utils ]
    ut_vals = [ np.average(uts) for (p, uts) in given_utils ]
    fig.add_trace(go.Scatter(x=p_vals,
        y=ut_vals, mode='lines+markers', hoverinfo='skip',
        name='avg social welfare', marker=dict(color='Blue')))
    
    #Add individual points trace
    indiv_p_vals = []
    indiv_ut_vals = []
    for (p, uts) in given_utils:
        indiv_ut_vals.extend(uts)
        indiv_p_vals.extend([p] * len(uts))
    fig.add_trace(go.Scatter(x=indiv_p_vals,
        y=indiv_ut_vals, mode='markers', hoverinfo='skip', opacity=0.7,
        marker=dict(color='LightSkyBlue'), name='social welfare per p'))


    fig.layout.update(showlegend=False)
    title_fmt = "Average Social Welfare (Normalized) vs p (r={0})"
    plot_title = title_fmt.format(r)
    fig.layout.update(title=plot_title, showlegend=True,
        xaxis=dict(title='p'), yaxis=dict(title='avg social welfare'),
        plot_bgcolor='rgba(0,0,0,0)')

    iplot(fig)

In [5]:
def run_sim(graph_func, plaw_resources=False):
    for r in [ 2 ** k for k in range(1, 8) ][::-1]:
        social_utils = []
        for p in np.linspace(1, 0, 10, endpoint=False):
            utils = []
            num_iter = 10
            for i in range(num_iter):
                G = create_sbm(p, r)
                
                if plaw_resources:
                    powerlaw_dist_time(G, 2)
                
                #Initialize even alloc strategy
                even_strat = EvenAlloc()
                even_strat.initialize_model(G)

                sim_g, sim_utils = run_simulation(G, even_strat)

                #Get global social welfare at end of simulation normalized by size
                utils.append(sum(sim_utils[-1]) / len(G.vertices))
            social_utils.append((p, utils))

        plot_util_comp(social_utils, r)

## Even resource distribution

In [6]:
run_sim(create_sbm)

## Power law resource distribution

In [7]:
run_sim(create_sbm, True)

KeyboardInterrupt: 